In [1]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.array as da
from numpy import cov
from numpy import std
from sklearn.metrics.pairwise import cosine_similarity
import random

# User User based Collaborative filtering

In [2]:
def init_spark():
    spark = SparkSession \
        .builder \
        .config("spark.driver.memory", "15g") \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [3]:
spark = init_spark()

KeyboardInterrupt: 

In [4]:
df = spark.read.text("D:/Courses/big data/project/Anime/rating.csv").rdd
data = df.map(lambda x: x[0].split(','))
data2 = data.groupBy(lambda x : x[0])

In [143]:
data3 = data2.collect()

In [145]:
utility = {}
all_animes = []
all_users = []
for user in range(len(data3)):
    if data3[user][0] == 'user_id':
        continue
    utility[int(data3[user][0])] = {}
    if int(data3[user][0]) not in all_users:
        all_users.append(int(i[0]))
    for i in data3[user][1]:        
        if int(i[1]) not in all_animes:
            all_animes.append(int(i[1]))
        if int(i[2]) == -1:
            continue            
        utility[int(i[0])][int(i[1])] = int(i[2])

In [148]:
all_animes

[853,
 3457,
 4224,
 6127,
 7054,
 9366,
 10408,
 11111,
 11553,
 11701,
 11757,
 11771,
 14713,
 16417,
 17513,
 18507,
 19315,
 19671,
 20507,
 21995,
 22319,
 25681,
 28223,
 30503,
 32729,
 19,
 63,
 98,
 99,
 101,
 120,
 147,
 149,
 150,
 226,
 227,
 237,
 243,
 322,
 431,
 552,
 857,
 863,
 874,
 990,
 1195,
 1313,
 1530,
 1535,
 1554,
 1571,
 1689,
 1696,
 1699,
 1721,
 1818,
 2034,
 2167,
 2236,
 2603,
 2924,
 2966,
 3230,
 4246,
 4896,
 5150,
 5681,
 6746,
 6880,
 8425,
 9989,
 10418,
 10721,
 10798,
 11433,
 121,
 122,
 135,
 269,
 877,
 969,
 1222,
 3092,
 3470,
 3731,
 4081,
 4722,
 4752,
 4814,
 4898,
 5678,
 5682,
 5835,
 6045,
 6164,
 6586,
 6707,
 6956,
 7724,
 9074,
 9289,
 9441,
 9513,
 9656,
 9919,
 9926,
 10460,
 10521,
 10719,
 10800,
 11013,
 11123,
 11617,
 1,
 5,
 29,
 30,
 31,
 32,
 33,
 43,
 44,
 45,
 46,
 47,
 49,
 57,
 66,
 67,
 71,
 72,
 73,
 97,
 117,
 118,
 123,
 145,
 153,
 155,
 164,
 170,
 180,
 182,
 189,
 190,
 191,
 192,
 194,
 198,
 199,
 201,
 205

In [93]:
del data3

In [11]:
import gc
gc.collect()

0

In [175]:
def sim2(matrix, user1, user2):
    matched = []
    for anime in matrix[user1].keys():
        if anime in matrix[user2].keys():
            matched.append(anime) 
    x = []
    y = []    
    for anime in matched:
        x.append(matrix[user1][anime])
        y.append(matrix[user2][anime])        

    f = (lambda i, j: i - j)
    x = f(x, np.mean(x))
    y = f(y, np.mean(y))
    #Pearson Correlation Coefficient
    numerator = (lambda x, y : x*y)
    denominator = (lambda x: x*x)
    d = math.sqrt(sum(denominator(x))) * math.sqrt(sum(denominator(y)))
    if d == 0:
        return 0
    similarity = sum(numerator(x, y))/ d
    return similarity

In [174]:
def most_similar_user(matrix, user, n=50):
    similarity = {}
    for p in matrix.keys():
        similarity[p] = sim2(matrix, user, p)
    sorted_list = sorted(similarity.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]     

In [176]:
def recommend(matrix, user):
    similar_users = most_similar_user(matrix, user)
    predictions = {}
    for anime in all_animes:
        num = 0
        den = 0
        for user in similar_users:
            if anime in matrix[user[0]].keys():
                num += matrix[user[0]][anime] * user[1]
                den += user[1]
        if den == 0:
            continue
        if anime in matrix[user[0]].keys():
            predictions[anime] = float(num/den)
    return predictions

In [189]:
# Dividing data into 80-20 split
#train = int(0.8 * len(all_users))
test= int(0.05 * len(all_users))
test_data_user = []
for _ in range(50):
    random_user = random.randint(1, len(all_users))
    if all_users[random_user] not in test_data_user:
        test_data_user.append(all_users[random_user])

In [191]:
test_data_user

[55457,
 6898,
 26058,
 45405,
 73420,
 30291,
 51630,
 64328,
 37384,
 36204,
 33222,
 35154,
 22338,
 11287,
 56021,
 46310,
 27209,
 10805,
 10113,
 59334,
 57811,
 46471,
 30416,
 44044,
 64290,
 29468,
 15361,
 25249,
 32842,
 65693,
 52917,
 69048,
 55715,
 43960,
 19496,
 26026,
 64085,
 47117,
 7229,
 15917,
 67708,
 18300,
 58141,
 53727,
 10074,
 37352,
 45412,
 70351,
 1100,
 12700]

In [183]:
test_animes = int(0.2 * len(all_animes))
test_data_animes = []
for _ in range(test_animes):
    random_anime = random.randint(1, len(all_animes))
    if all_animes[random_anime] not in test_data_animes:
        test_data_animes.append(all_users[random_anime])

In [184]:
import copy
training_data = copy.deepcopy(utility)

In [185]:
for user in test_data_user:
    for anime in test_data_animes:
        if anime in training_data[user].keys():        
            del training_data[user][anime]                        

In [186]:
def start():
    predicted = {}
    for user in test_data_user:
        predicted[user] = recommend(training_data, user)
        print(user)
    return predicted

In [192]:
predicted = start()

55457
6898
26058
45405
73420
30291
51630
64328
37384
36204
33222
35154
22338
11287
56021
46310
27209
10805
10113
59334
57811
46471
30416
44044
64290
29468
15361
25249
32842
65693
52917
69048
55715
43960
19496
26026
64085
47117
7229
15917
67708
18300
58141
53727
10074
37352
45412
70351
1100
12700


In [195]:
RMSE(utility, predicted)

1.092775318073847

In [196]:
print(utility[1100])

{1: 10, 6594: 10, 3784: 8, 777: 10, 330: 7, 1292: 7, 269: 7, 14349: 8, 2001: 8, 6547: 8, 889: 9, 6746: 8, 5114: 10, 30: 10, 6880: 7, 227: 10, 164: 10, 9253: 9, 16742: 9, 11111: 8, 2025: 9, 7338: 7, 813: 10, 16498: 8, 13331: 8, 121: 9, 5177: 9, 1210: 8, 10620: 8, 12413: 8, 1535: 6}


In [197]:
print(predicted[1100])

{1: 9.666666666666666, 5: 9.0, 6: 8.75, 47: 10.0, 522: 9.0, 1303: 9.0, 1292: 8.0, 270: 8.428571428571429, 400: 9.0, 2961: 7.499999999999998, 1943: 9.2, 537: 7.0, 1051: 10.0, 474: 9.0, 30: 9.555555555555555, 543: 9.0, 32: 9.75, 33: 9.0, 164: 9.833333333333332, 679: 7.0, 552: 7.999999999999998, 937: 9.0, 43: 8.75, 431: 9.555555555555555, 986: 7.0, 306: 8.0, 567: 9.0, 570: 8.0, 31: 9.2, 1802: 7.0, 578: 9.5, 2273: 10.0, 199: 9.416666666666666, 974: 10.0, 81: 10.0, 339: 8.666666666666666, 1494: 10.0, 87: 8.0, 2265: 7.0, 90: 9.0, 731: 10.0, 732: 10.0, 1117: 7.666666666666667, 96: 8.0, 528: 7.666666666666667, 227: 9.25, 617: 8.0, 1264: 9.0, 2281: 9.0, 1016: 9.5, 763: 8.0, 764: 9.0, 894: 6.999999999999999}


In [194]:
def RMSE(actual, predicted):
    error = 0
    n = 0
    for user in test_data_user:
        for anime in test_data_animes:
            if anime in predicted[user].keys() and anime in actual[user].keys():
                error += (actual[user][anime] - predicted[user][anime])**2
                n += 1
    return math.sqrt(error/n)

In [198]:
def recommend_animes(for_user, n = 10):
    predictions = recommend(training_data, for_user)
    sorted_list = sorted(predictions.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]

In [200]:
recommended_list = recommend_animes(5)

In [120]:
df = pd.read_csv("D:/Courses/big data/project/Anime/anime.csv")


In [201]:
for anime in recommended_list:
    values = df[['anime_id','name', 'genre', 'type']].loc[df['anime_id']==anime[0]]
    print(str(values['anime_id']) + "," + str(values['name']) + "," + str(values['genre']) + "," + str(values['type']) + "\n")

282    2034
Name: anime_id, dtype: int64,282    Lovely★Complex
Name: name, dtype: object,282    Comedy, Romance, Shoujo
Name: genre, dtype: object,282    TV
Name: type, dtype: object

131    4224
Name: anime_id, dtype: int64,131    Toradora!
Name: name, dtype: object,131    Comedy, Romance, School, Slice of Life
Name: genre, dtype: object,131    TV
Name: type, dtype: object

267    8129
Name: anime_id, dtype: int64,267    Kuragehime
Name: name, dtype: object,267    Comedy, Josei, Slice of Life
Name: genre, dtype: object,267    TV
Name: type, dtype: object

2335    5041
Name: anime_id, dtype: int64,2335    Guin Saga
Name: name, dtype: object,2335    Adventure, Fantasy
Name: genre, dtype: object,2335    TV
Name: type, dtype: object

24    164
Name: anime_id, dtype: int64,24    Mononoke Hime
Name: name, dtype: object,24    Action, Adventure, Fantasy
Name: genre, dtype: object,24    Movie
Name: type, dtype: object

71    578
Name: anime_id, dtype: int64,71    Hotaru no Haka
Name: name, dty